# Test Spherical Signal Generation

## Description
This notebook provides a test script to generate the spherical features for input. 

In [2]:
from data_source import DataSource
from visualize import Visualize
from sphere import Sphere
import sys
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import open3d as o3d
from mission_indices import MissionIndices
from database_parser import DatabaseParser

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Load the spherical image

In [2]:
ds = DataSource('')
#sph_image_cart = ds.loadPointCloudFromPath('/tmp/training/training_anchor_sph_images/000000000000001.ply')
cloud = ds.loadPointCloudFromPath('/tmp/training/training_anchor_pointclouds/000000000000001.ply')
#img_feat = np.reshape(sph_image_cart.transpose(), (4,200,200))
print('Original shape ', cloud.shape)
T = np.array(
        [[0.9997770641962381,  0.016299234847643,  -0.01344410000000, 0.0331989],
         [-0.016286738363819,   0.999867935740224,  0.001094290000000, -0.121313],
         [0.01346018117580,   -0.00087509685356,   0.999909000000000, 0.15737],
         [0, 0, 0, 1]])


pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
pcd.transform(T)
dst = np.asarray(pcd.points)
transformed = np.column_stack((dst, cloud[:,3]))
print('Original shape ', transformed.shape)

viz = Visualize()
sphere = Sphere(transformed)
#viz.visualizeSphere(sphere, True)
viz.visualizeRawPointCloud(sphere, False)

Original shape  (131072, 4)
Original shape  (131072, 4)


In [97]:
sphere = Sphere(sph_image_cart)
viz = Visualize()
#viz.visualizeCartesianSphere(sph_image_cart, False)
viz.visualizeSphere(sphere, True)

In [111]:
sphere = Sphere(sph_image_cart)
cart_sphere = sphere.getProjectedInCartesian()
cart_sphere.shape
#sphere.intensity.shape
#np.column_stack((cart_sphere, sphere.intensity))
#test = np.append(cart_sphere, sphere.intensity, axis=1)

(40000, 3)

In [125]:
from plyfile import PlyData, PlyElement
plydata = PlyData.read('/tmp/training/training_anchor_pointclouds/000000000000001.ply')
'scalar2' in plydata['vertex']._property_lookup

AttributeError: 'PlyData' object has no attribute 'shape'

In [87]:
test = np.zeros((3, 200, 200))
f12 = np.ones((2,200,200))*1
f3 = np.ones((1, 200, 200))*2

test[0,:,:] = f12[0,:,:]
test[1,:,:] = f12[1,:,:]
test[2,:,:] = f3[0,:,:]


3

In [93]:
features = [np.zeros((3, 200, 200))]*10
features[0][1,1,1]

0.0

In [6]:
dataset_path = "/mnt/data/datasets/Spherical/test_training/"
db_parser = DatabaseParser(dataset_path)

training_missions, test_missions = MissionIndices.get_arche_low_res()
training_indices, test_indices = db_parser.extract_training_and_test_indices(
    training_missions, test_missions)

n_test_data = 8000
n_test_cache = n_test_data
ds_test = DataSource(dataset_path, n_test_cache, -1)
idx = np.array(training_indices['idx'].tolist())
ds_test.load(n_test_data, idx)

Reading missions db from /mnt/data/datasets/Spherical/test_training/missions.csv
Read 52 entries.



Loading anchors from:	/mnt/data/datasets/Spherical/test_training//training_anchor_pointclouds/ and /mnt/data/datasets/Spherical/test_training//training_anchor_sph_images/



Loading positives from:	/mnt/data/datasets/Spherical/test_training//training_positive_pointclouds/ and /mnt/data/datasets/Spherical/test_training//training_positive_sph_images/



Loading negatives from:	/mnt/data/datasets/Spherical/test_training//training_negative_pointclouds/ and /mnt/data/datasets/Spherical/test_training//training_negative_sph_images/



Done loading dataset.
	Anchor point clouds total: 	52
	Anchor images total: 		52
	Anchor poses total: 		52
	Positive point clouds total: 	52
	Positive images total: 		52
	Positive poses total: 		52
	Negative point clouds total: 	52
	Negative images total: 		52
	Negative poses total: 		52


In [59]:
ds_test.filterClusters(ds_test.anchor_poses, ds_test.positive_poses)

Minimum distance is 1.5213253674727174 at index 0 for sample 15
[[ 66.3513   -34.8314    -0.351782]
 [ 66.5754   -36.2498     0.150564]]
[0, 15]
Minimum distance is 1.5011588001543974 at index 1 for sample 47
[[ 66.3513   -34.8314    -0.351782]
 [ 66.5754   -36.2498     0.150564]
 [ 65.4      -37.183      0.118155]]
[0, 15, 47]
Minimum distance is 27.16334466582634 at index 2 for sample 50
[[ 66.3513    -34.8314     -0.351782 ]
 [ 66.5754    -36.2498      0.150564 ]
 [ 65.4       -37.183       0.118155 ]
 [ 80.2366    -59.9365      0.0893171]]
[0, 15, 47, 50]
Minimum distance is 7.777646323674765 at index 3 for sample 51
[[ 66.3513    -34.8314     -0.351782 ]
 [ 66.5754    -36.2498      0.150564 ]
 [ 65.4       -37.183       0.118155 ]
 [ 80.2366    -59.9365      0.0893171]
 [ 76.4088    -53.1668      0.193679 ]]
[0, 15, 47, 50, 51]


In [41]:
x1 = np.arange(9.0).reshape((3, 3))
x2 = np.arange(3.0)
#print(x1)
#print(x2)

x3 = np.subtract(x1, x2)
#print(x3)
distances = np.linalg.norm(x3, axis=1)
min_dist = np.amin(distances)
min_idx = np.where(distances == min_dist)

print(f'Minimum distance is {min_dist} at index {min_idx[0][0]}')

Minimum distance is 0.0 at index 0


In [56]:
non_clustered = [ds_test.anchor_poses[0,5:8]]
non_clustered_indices = [0]
print(np.array(non_clustered))
for i in range(1,50):
    curr_pose = ds_test.anchor_poses[i, 5:8]
    diff = np.subtract(np.array(non_clustered), curr_pose)
    distances = np.linalg.norm(diff, axis=1)
    min_dist = np.amin(distances)
    if min_dist < 0.3:
        continue
            
    min_idx = np.where(distances == min_dist)
    print(f'Minimum distance is {min_dist} at index {min_idx[0][0]} for sample {i}')
    non_clustered.append(curr_pose)
    non_clustered_indices.append(i)
print(np.array(non_clustered))
print(non_clustered_indices)
assert len(non_clustered) == len(non_clustered_indices)

[[ 66.3513   -34.8314    -0.351782]]
Minimum distance is 1.5213253674727174 at index 0 for sample 15
Minimum distance is 0.3290786636960846 at index 1 for sample 23
Minimum distance is 0.3209681953153622 at index 2 for sample 38
Minimum distance is 0.306612709979544 at index 3 for sample 43
Minimum distance is 0.48132905630244005 at index 4 for sample 46
[[ 66.3513   -34.8314    -0.351782]
 [ 66.5754   -36.2498     0.150564]
 [ 66.3882   -36.5203     0.141694]
 [ 66.1596   -36.7455     0.148592]
 [ 65.8951   -36.8982     0.121503]
 [ 65.4722   -37.128      0.126524]]
[0, 15, 23, 38, 43, 46]
